In [6]:
from mesa import Agent, Model
from mesa.space import NetworkGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import networkx as nx
import graph_init
import pandas as pd
import station as st

import network_example as ne
import numpy as np
import pathfinding as pf

In [9]:
class Commuter(Agent):
    def __init__(self, model, current_pos, distance_left, intermediate_node, destination):
        super().__init__(model)
        self.destination = destination
        self.intermediate_node = intermediate_node
        self.distance_left = distance_left
        self.current_pos = current_pos
        self.biking = False

        assert self.model.grid.G.nodes[self.current_pos]['type'] == 'destination', "Agent must start at a destination"
        
    def step(self):
        self.current_pos = self.model.get_node(self)  # Update the agent's current position
        if self.distance_left <= 0:
            # If the agent has reached the intermediate node, set new start node
            self.current_pos = self.intermediate_node
            self.model.grid.move_agent(self, self.current_pos)

            # Return bike if the agent is at the intended station and there are spots available
            if self.biking and self.model.grid.G.nodes[self.current_pos]['type'] == 'station' and self.model.grid.G.nodes[self.current_pos]['data'].get_spot_availability():
                self.model.grid.G.nodes[self.current_pos]['data'].return_bike()
                self.biking = False

            # If the agent has reached the destination, set new destination
            # Else, the agent will rent a bike if it is at a station and there are bikes available
            if self.current_pos == self.destination:
                self.destination = self.model.sample_destination(self.current_pos)
            elif not(self.biking) and self.model.grid.G.nodes[self.current_pos]['type'] == 'station' and self.model.grid.G.nodes[self.current_pos]['data'].get_bike_availability():
                self.model.grid.G.nodes[self.current_pos]['data'].rent_bike()
                self.biking = True

            # Set new intermediate node
            if self.model.grid.G.nodes[self.current_pos]['type'] == 'destination':
                path = pf.pathfind(self.model.grid.G, self.current_pos, self.destination, False)
            else:
                bike = self.model.grid.G.nodes[self.current_pos]['data'].get_bike_availability()
                path = pf.pathfind(self.model.grid.G, self.current_pos, self.destination, bike)

            self.intermediate_node = path[0]

            if path[1]:
                self.distance_left = self.model.grid.G[self.current_pos][self.intermediate_node]['weight']
            else:
                self.distance_left = self.model.grid.G[self.current_pos][self.intermediate_node]['weight'] * self.model.walking_multiplier
        else:
            self.distance_left -= 1        

    
    def get_agent_position(self):
    # needed seperate function for data collector
        return self.current_pos
    
    def get_distance_left(self):
    # needed seperate function for data collector
        return self.distance_left
    
    def get_agent_destination(self):
        return self.destination
    
    def get_intermediate_node(self):
        return self.intermediate_node


class MyModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        g = ne.basic_graph()[0]
        self.walking_multiplier = 10
        self.grid = NetworkGrid(g)
        # self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            agent_reporters={"Position": lambda agent: agent.get_agent_position(),
                             "Distance_Left": lambda agent: agent.get_distance_left(),
                             "Intermediate_Node": lambda agent: agent.get_intermediate_node(),
                             "Destination": lambda agent: agent.get_agent_destination()})
        
        self.stations = pf.get_stations(self.grid.G)
        self.destinations = pf.get_destinations(self.grid.G)
        self.destination_w = ne.basic_weights()

        # Create agents and place them on the grid
        for i in range(1, n_agents + 1):
            node_id = self.random.choice(self.destinations)

            destination_node = self.sample_destination(node_id)
            path = pf.pathfind(self.grid.G, node_id, destination_node, False)
            intermediate_node = path[0]
            distance_left = self.grid.G[node_id][intermediate_node]['weight'] * self.walking_multiplier

            commuter = Commuter(self, current_pos=node_id, distance_left=distance_left, intermediate_node=intermediate_node, destination=destination_node)
            # self.schedule.add(commuter)
            self.grid.place_agent(commuter, node_id)
            
        self.datacollector.collect(self)
    
    def get_node(self, agent):
        # Iterate over all node IDs in the graph
        for node_id in self.grid.G.nodes():
            # Get the agents at the current node
            agents_at_node = self.grid.get_cell_list_contents([node_id])
            # If the given agent is found at this node, return the node ID
            if agent in agents_at_node:
                return node_id
        return None  # If the agent is not found in any node
    
    def sample_destination(self, current_pos):
        while True:
            destination = np.random.choice(self.destinations, p=ne.basic_weights())
            if destination != current_pos:
                return destination

    def step(self):
        # print("Model step")
        # self.schedule.step()
        self.agents.shuffle_do("step")
        self.datacollector.collect(self)

In [10]:
model = MyModel(20)
for i in range(100):
    print(model.agents[0].get_agent_position(), model.agents[0].get_distance_left(), model.agents[0].get_agent_destination(), model.agents[0].get_intermediate_node())
    model.step()

agent_data = model.datacollector.get_agent_vars_dataframe()
agent_id = 8
agent_specific_data = agent_data[agent_data.index.get_level_values('AgentID') == agent_id]

# Print the filtered DataFrame
# print(agent_specific_data)

4 120.0 5 5
4 119.0 5 5
4 118.0 5 5
4 117.0 5 5
4 116.0 5 5
4 115.0 5 5
4 114.0 5 5
4 113.0 5 5
4 112.0 5 5
4 111.0 5 5
4 110.0 5 5
4 109.0 5 5
4 108.0 5 5
4 107.0 5 5
4 106.0 5 5
4 105.0 5 5
4 104.0 5 5
4 103.0 5 5
4 102.0 5 5
4 101.0 5 5
4 100.0 5 5
4 99.0 5 5
4 98.0 5 5
4 97.0 5 5
4 96.0 5 5
4 95.0 5 5
4 94.0 5 5
4 93.0 5 5
4 92.0 5 5
4 91.0 5 5
4 90.0 5 5
4 89.0 5 5
4 88.0 5 5
4 87.0 5 5
4 86.0 5 5
4 85.0 5 5
4 84.0 5 5
4 83.0 5 5
4 82.0 5 5
4 81.0 5 5
4 80.0 5 5
4 79.0 5 5
4 78.0 5 5
4 77.0 5 5
4 76.0 5 5
4 75.0 5 5
4 74.0 5 5
4 73.0 5 5
4 72.0 5 5
4 71.0 5 5
4 70.0 5 5
4 69.0 5 5
4 68.0 5 5
4 67.0 5 5
4 66.0 5 5
4 65.0 5 5
4 64.0 5 5
4 63.0 5 5
4 62.0 5 5
4 61.0 5 5
4 60.0 5 5
4 59.0 5 5
4 58.0 5 5
4 57.0 5 5
4 56.0 5 5
4 55.0 5 5
4 54.0 5 5
4 53.0 5 5
4 52.0 5 5
4 51.0 5 5
4 50.0 5 5
4 49.0 5 5
4 48.0 5 5
4 47.0 5 5
4 46.0 5 5
4 45.0 5 5
4 44.0 5 5
4 43.0 5 5
4 42.0 5 5
4 41.0 5 5
4 40.0 5 5
4 39.0 5 5
4 38.0 5 5
4 37.0 5 5
4 36.0 5 5
4 35.0 5 5
4 34.0 5 5
4 33.0 5 5
4 32.0 5 5
